# 第2章　分散表現

## 2.2 cos類似度

### cosine_simirarityモジュールによるcos類似度

In [2]:
# CountVectorizerモジュールの入力形式に変換する関数
import janome
from janome.tokenizer import Tokenizer

t = Tokenizer()

def make_corpus(documents):
    result_corpus = []
    
    for adocument in documents:
        words = [token for token in t.tokenize(adocument,wakati=True)]
        text = " ".join(words)
        result_corpus.append(text)
    return result_corpus

In [3]:
document1 = '私は秋田犬が大好きです。私の好きな犬は秋田犬です。'
document2 = '私は犬が苦手です。犬より猫のほうが好きです。'
document3 = '私は本が好きです。毎日本を読みます。'
document4 = '私は数学が苦手です。'
document5 = '私は運動が得意で、走るのが速いです。'
document6 = '私は毎日8時間寝ています。'
document7 = '私は肉と魚が好きです。嫌いな食べ物はピーマンです。'
document8 = '私は毎朝電車で通勤しています。満員電車は嫌いです。'
document9 = '今日はいい天気です。日差しがまぶしい。'
document10 = '明日は雨が降るかもしれません。'

documents = [document1,document2,document3,document4,document5,document6,document7,document8,document9,document10]
dict = make_corpus(documents) 
print(dict)


['私 は 秋田 犬 が 大好き です 。 私 の 好き な 犬 は 秋田 犬 です 。', '私 は 犬 が 苦手 です 。 犬 より 猫 の ほう が 好き です 。', '私 は 本 が 好き です 。 毎 日本 を 読み ます 。', '私 は 数学 が 苦手 です 。', '私 は 運動 が 得意 で 、 走る の が 速い です 。', '私 は 毎日 8 時間 寝 て い ます 。', '私 は 肉 と 魚 が 好き です 。 嫌い な 食べ物 は ピーマン です 。', '私 は 毎朝 電車 で 通勤 し て い ます 。 満員 電車 は 嫌い です 。', '今日 は いい 天気 です 。 日差し が まぶしい 。', '明日 は 雨 が 降る かも しれ ませ ん 。']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b',sublinear_tf=True)
X = vectorizer.fit_transform(dict)
X.shape

(10, 55)

In [6]:
#次元数を減らしてみる
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=3,n_iter=5,random_state=42)
newX = svd.fit_transform(X)
print(newX)
print(newX.shape)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

[[ 0.6755768  -0.27843101 -0.26856757]
 [ 0.71449893 -0.3433819  -0.1981751 ]
 [ 0.46417642  0.19180123 -0.02346579]
 [ 0.60150519 -0.11702002  0.07424716]
 [ 0.50587724 -0.02412961  0.11104759]
 [ 0.26155699  0.7271639  -0.07386861]
 [ 0.53685042  0.01953388 -0.04135825]
 [ 0.37314351  0.67744546 -0.02297382]
 [ 0.30515288 -0.07094008  0.48275125]
 [ 0.16987888 -0.02828892  0.79982474]]
(10, 3)
[0.03800946 0.15283318 0.12135128]
0.3121939206181686


In [10]:
# cos類似度を求める
from sklearn.metrics.pairwise import cosine_similarity
A = newX[0]
B = newX[9]
print(A)
print(A.reshape(1,-1))
print(cosine_similarity(A.reshape(1,-1),B.reshape(1,-1)))

[ 0.6755768  -0.27843101 -0.26856757]
[[ 0.6755768  -0.27843101 -0.26856757]]
[[-0.1447005]]


In [12]:
# cosine_similarityの第2引数を省略
# cos類似度を求める
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(newX)
print(result)
result.shape

[[ 1.          0.99277153  0.6808168   0.87194947  0.78945428 -0.00980307
   0.87817903  0.11560315  0.21484288 -0.1447005 ]
 [ 0.99277153  1.          0.65826394  0.90257702  0.82075836 -0.07594427
   0.87463065  0.06095802  0.31200454 -0.0410048 ]
 [ 0.6808168   0.65826394  1.          0.82169839  0.87300438  0.67282425
   0.93728093  0.78059133  0.40334546  0.1328746 ]
 [ 0.87194947  0.90257702  0.82169839  1.          0.9853611   0.13930184
   0.95484607  0.3003854   0.640971    0.32647966]
 [ 0.78945428  0.82075836  0.87300438  0.9853611   1.          0.26476365
   0.95403579  0.42341943  0.70274471  0.41357575]
 [-0.00980307 -0.07594427  0.67282425  0.13930184  0.26476365  1.
   0.37697738  0.98542669 -0.01663172 -0.0554525 ]
 [ 0.87817903  0.87463065  0.93728093  0.95484607  0.95403579  0.37697738
   1.          0.51453083  0.4594673   0.13059253]
 [ 0.11560315  0.06095802  0.78059133  0.3003854   0.42341943  0.98542669
   0.51453083  1.          0.12287832  0.04083389]
 [ 0.214

(10, 10)

### PyTorchによるcos類似度の実装

In [1]:
!pip3 install torch torchvision torchaudio

In [14]:
import torch
print(torch.__version__)

1.12.1+cpu


In [2]:
import torch
import torch.nn.functional as F
A = torch.FloatTensor(newX[0])
B = torch.FloatTensor(newX[9])
F.cosine_similarity(A,B,dim=0)

NameError: name 'newX' is not defined

## 2.3 word2vec

### gensimモジュールによるword2vecの作成

In [3]:
# 分かち書きファイルの作成
import janome
from janome.tokenizer import Tokenizer

t = Tokenizer()

datafile = "melos.txt"
wakatifile = "wakati.txt"

with open(wakatifile,'w',encoding='UTF-8') as f2:
    with open(datafile,'r',encoding='UTF-8') as f1:
        for line in f1:
            for token in t.tokenize(line,wakati=True):
                f2.write(token + " ")
            f2.write('\n')

In [9]:
from gensim.models import word2vec
from gensim.models import Word2Vec
sentences = word2vec.LineSentence(wakatifile)
model = Word2Vec(sentences)
print(type(model))

<class 'gensim.models.word2vec.Word2Vec'>


### 事前学習済のword2vec

In [1]:
from gensim.models import KeyedVectors

model = KeyedVectors.load('chive-1.1-mc5-aunit_gensim\chive-1.1-mc5-aunit.kv')

In [2]:
model.similarity('葡萄','メロン')

0.50058043

In [3]:
len(model.key_to_index)

322094

In [4]:
print(model.most_similar('葡萄',topn=5))

[('巨峰', 0.7231999635696411), ('果実', 0.6389086842536926), ('桜ん坊', 0.6240198016166687), ('シャルドネ', 0.618291437625885), ('ワイン', 0.6176699995994568)]


In [13]:
model.most_similar(positive=['山','夏'],negative=['海'],topn=3)

[('秋', 0.5729417204856873),
 ('冬', 0.5290267467498779),
 ('春', 0.527359127998352)]

### 2.4 doc2vec

In [16]:
from gensim.models.doc2vec import TaggedDocument
with open('wakati_doc.txt', encoding='utf-8') as f:
    docs = [TaggedDocument(words=data.split(),tags=[i]) for i,data in enumerate(f)]

In [17]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec(docs)

In [18]:
model.save('mymodel.model')
medel = Doc2Vec.load('mymodel.model')

In [20]:
model.dv[0]

array([-1.08137354e-02, -3.60686448e-03, -1.68751404e-02,  1.00992694e-02,
        2.70028273e-03, -2.33986657e-02, -1.93522498e-03,  1.79511961e-02,
       -1.64268762e-02, -6.23647822e-03, -6.03719335e-03, -1.80389471e-02,
       -2.55854242e-03,  3.33547505e-04, -6.27460191e-03, -1.80697627e-02,
        1.20822592e-02, -4.17566299e-03, -1.06561501e-02, -3.03472783e-02,
        1.20246354e-02,  1.37604645e-03,  5.35555743e-03, -6.29999861e-03,
        6.32416923e-03, -1.41887018e-03, -2.22760793e-02, -1.37436436e-02,
       -9.75212734e-03, -1.32230837e-02,  1.26991915e-02,  1.62335671e-02,
       -3.38499347e-04, -1.54851247e-02, -1.25121120e-02,  2.70942431e-02,
       -6.13774732e-03, -2.54533496e-02, -1.35383690e-02, -7.05700414e-03,
        3.49374046e-03, -2.63165459e-02, -8.77705534e-05, -1.61053482e-02,
        7.55836256e-03, -2.26655193e-02, -9.27963760e-03,  2.73518241e-03,
        1.23287337e-02, -4.61329194e-03,  9.55367181e-03, -2.19134502e-02,
       -4.44071181e-03, -

In [23]:
model.dv.most_similar(1,topn=4)

[(731, 0.9468039274215698),
 (576, 0.9417536854743958),
 (196, 0.9413872957229614),
 (573, 0.9384422302246094)]

In [24]:
newdoc = ['私','は','秋田','犬','が','大好き','。']
model.infer_vector(newdoc)

array([-0.00859902,  0.01011155, -0.00808303, -0.0039235 , -0.00041212,
       -0.01831308,  0.0063673 ,  0.02537156, -0.00326623, -0.00473055,
       -0.00889074, -0.02238915, -0.00067427,  0.00557083, -0.00108072,
       -0.00913868,  0.01139112, -0.00185002,  0.00279234, -0.02671054,
        0.01507623, -0.00178387,  0.01703887, -0.00687599,  0.00273824,
        0.0147821 , -0.02003526, -0.00321776, -0.01605098, -0.00430303,
        0.00124331,  0.00877965,  0.00949923, -0.00797731, -0.01230116,
        0.02930696, -0.01092273, -0.01771256, -0.01616312, -0.00896365,
        0.00668247, -0.0205541 , -0.00510602, -0.01101989,  0.00432243,
       -0.02175647, -0.00861292,  0.004426  ,  0.00603218,  0.00059241,
        0.0085566 , -0.02613855, -0.00027886,  0.01040063, -0.0080724 ,
        0.00665825,  0.01113282, -0.01233829, -0.01430307,  0.0183921 ,
        0.00768219, -0.02199128,  0.02865104, -0.00252148, -0.02320654,
        0.01620888,  0.00626117,  0.01115923, -0.02461884,  0.00